# Time series LSTM

### Setup notebook

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim

%matplotlib inline

In [ ]:
# set random seed to 0
np.random.seed(0)
torch.manual_seed(0)

### Data

Create synthetic data file (No Noise):

In [ ]:
x = np.arange(5).reshape(-1, 1)
x.reshape(-1, 1)
r = np.random.randint(-5, 5, 5)
r.reshape(-1, 1)

# Broadcasting to random shifted range
x + r

#### Synthetic time series

In [ ]:
# Dimensions
T = 20   # amplitude range
L = 1000 # features, # inputs, shifted ranges, time steps
N = 100  # examples & labels

In [ ]:
x = np.empty((N, L), 'int64')
L_rand = np.random.randint(-4*T, 4*T, N)
noise = np.random.randn(N*L) * 0.3

# Broadcast random over range
x[:] = np.arange(L) + L_rand.reshape(N, 1)
data = np.sin(x/T).astype('float64')
data += noise.reshape(N, -1)
data

#### Save and load pytorch data file

In [ ]:
torch.save(data, open('traindata.pt', 'wb'))

In [ ]:
data = torch.load('traindata.pt')

#### Train test sets

In [ ]:
# Train set [3:]: X_train = inputs = [:-1], y_train = outputs = [:1]
inputs = Variable(torch.from_numpy(data[3:, :-1]), requires_grad=False)
target = Variable(torch.from_numpy(data[3:, 1:]), requires_grad=False)

# Test set [:3]: X_test = inputs = [:3], y_test = outputs = [1:]
test_input = Variable(torch.from_numpy(data[:3, :-1]), requires_grad=False)
test_target = Variable(torch.from_numpy(data[:3, 1:]), requires_grad=False)

### Build model

In [ ]:
class Sequence(nn.Module):
    def __init__(self):
        super(Sequence, self).__init__()
        self.lstm1 = nn.LSTMCell(1, 51)
        self.lstm2 = nn.LSTMCell(51, 51)
        self.linear = nn.Linear(51, 1)

    def forward(self, inputs, future=0):
        outputs = []
        kwargs = {'torch.zeros(input.size(0), 51).double(), requires_grad=False'}
        
        h_t1 = Variable(torch.zeros(inputs.size(0), 51).double(), requires_grad=False)
        c_t1 = Variable(torch.zeros(inputs.size(0), 51).double(), requires_grad=False)
        h_t2 = Variable(torch.zeros(inputs.size(0), 51).double(), requires_grad=False)
        c_t2 = Variable(torch.zeros(inputs.size(0), 51).double(), requires_grad=False)

        for i, input_t in enumerate(inputs.chunk(inputs.size(1), dim=1)):
            h_t1, c_t1 = self.lstm1(input_t, (h_t1, c_t1))
            h_t2, c_t2 = self.lstm2(h_t1, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs += [output]
            
        for i in range(future):# if we should predict the future
            h_t1, c_t1 = self.lstm1(output, (h_t1, c_t1))
            h_t2, c_t2 = self.lstm2(h_t1, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs += [output]
            
        outputs = torch.stack(outputs, 1).squeeze(2)
        return outputs


In [ ]:
# build the model
seq = Sequence()
seq.double() # TODO

# loss
criterion = nn.MSELoss()

# use LBFGS as optimizer since we can load the whole data to train
optimizer = optim.LBFGS(seq.parameters(), lr=0.8)

### Train model

Using closure paradigm.

In [ ]:
for i in range(5):
    print(f'Iteration: {i}')
    
    def closure():
        optimizer.zero_grad()
        out = seq(inputs)
        loss = criterion(out, target)
        print(f' - loss: {loss.data.numpy()}')
        loss.backward()
        return loss
    
    _ = optimizer.step(closure)
    
print('Training done!')

### Predict

In [ ]:
future = 1000
pred = seq(test_input, future=future)
loss = criterion(pred[:, :-future], test_target)

print(f'test loss: {loss.data.numpy()}')
y = pred.data.numpy()

In [ ]:
inputs.size()

### Visualize prediction

In [ ]:
# draw the result
def draw(yi, color):
    _ = plt.figure(figsize=(20,8))
    _ = plt.title('Predict future values for time sequences\n(Dashlines are predicted values)', fontsize=30)
    _ = plt.xlabel('x', fontsize=16)
    _ = plt.ylabel('y', fontsize=16)
    _ = plt.xticks(fontsize=16)
    _ = plt.yticks(fontsize=16)
    _ = plt.plot(np.arange(inputs.size(1)), yi[:inputs.size(1)], color, linewidth=2.)
    _ = plt.plot(np.arange(inputs.size(1), inputs.size(1)+future), yi[inputs.size(1):], f'{color}:', linewidth=2.)
    plt.show();
    
draw(y[0], 'r')
draw(y[1], 'g')
draw(y[2], 'b')

# plt.savefig(f'predict_{i}.pdf')
# plt.close();